# Multiple Line Acquisition Apodization
Multiple Line Acquisition (MLA) lets us beamform multiple scanlines per transmit. In `vbeam`, MLA is implemented as a subclass of `vbeam.core.Apodization`.

_A brief summary of the `vbeam.core.Apodization` class is that it contains a single function `__call__`, which in Python means that we can call the object as if it was a function. Calling the `Apodization` object as a function returns the apodization weighting for a single sender, point, receiver, and transmit. This becomes more clear in the following examples._

Let's start by constructing our `MLAApodization` object. It takes two arguments: a window object (subclass of `vbeam.apodization.window.Window`) and a beam width. The window determines how to "smooth out" the scanlines and the beam width determines the "thickness" of the scanlines.

Let's start by defining an `MLAApodization` with a rectangular window (no "smoothing") and a beam width of 1.

In [ ]:
from vbeam.apodization import MLAApodization, Rectangular

apodization = MLAApodization(Rectangular(), 1)
print(apodization)


As mentioned, an `Apodization` object can be called as a function of a sender position, point position, receiver position, and the wave data for a given transmit. For wave data, `MLAApodization` requires the `"source"` (another word for focal point) field to be present.

Let's try to get the apodization weighting for a couple of point positions: one that is within the focused scanlines and one that is outside of the focused scanlines. We can assert that the former gets a weighting of 1 (since it is in focus) and the latter 0 (since it is outside of focus).

In [ ]:
import numpy as np
from vbeam.core import WaveData, ElementGeometry

focal_point = np.array([-1.0, 0.0, 2.5])  # (x, y, z)
point_within_focus = np.array([-1.0, 0.0, 1.0])
point_outside_focus = np.array([1.0, 0.0, 2.0])
sender = ElementGeometry(np.array([-1.0, 0.0, 0.0]), 0.0, 0.0)
receiver = ElementGeometry(np.array([0.0, 0.0, 0.0]), 0.0, 0.0)
wave_data = WaveData(source=focal_point)

assert apodization(sender, point_within_focus, receiver, wave_data) == 1
assert apodization(sender, point_outside_focus, receiver, wave_data) == 0


This is perhaps better understood visually. For this we have the `vbeam.apodization.visualize_apodization` function.

`visualize_apodization` takes the apodization object and the same arguments as when calling it directly, except it takes a `vbeam.scan.Scan` object instead of a point position. A `Scan` object defines (among a few other things) what points to image. `visualize_apodization` creates a visualization of the apodization function across all the points defined by scan.

Let's create a simple linear scan to visualize our `MLAApodization`.

In [ ]:
from vbeam.apodization import visualize_apodization
from vbeam.scan import linear_scan

scan = linear_scan(np.linspace(-2, 2), np.linspace(0, 5))
visualization = visualize_apodization(apodization, scan, sender, receiver, wave_data)
print(f"{visualization.shape=}")


In [ ]:
# Code to plot the visualization and some of the defined points
import matplotlib.pyplot as plt


def plot_point(ax, point, text, color=None):
    ax.scatter([point[0]], [point[2]], color=color)
    ax.text(point[0], point[2] + 0.3, text, color=color, horizontalalignment="center")


def plot_visualization(
    ax, visualization, focal_point=None, sender_position=None, extent=None
):
    ax.imshow(
        visualization.T, aspect="auto", extent=extent, origin="lower", cmap="gray"
    )
    plt.gca().invert_yaxis()
    if focal_point is not None and sender_position is not None:
        plot_point(ax, focal_point, "Focal point", "blue")
        plot_point(ax, sender_position, "Sender", "blue")


In [ ]:
plot_visualization(plt, visualization, focal_point, sender.position, scan.bounds)
plot_point(plt, point_within_focus, "Point in focus", "green")
plot_point(plt, point_outside_focus, "Point out of focus", "red")


`MLAApodization` will produce approximately the same result as scanline imaging if the beamwidth is equal to the `width/num_x` of the scan. In other words, setting the beamwidth to the delta-x of the scan means that the scanlines will have a width of one pixel.

In [ ]:
delta_x = scan.spacings[0]
visualization = visualize_apodization(
    MLAApodization(Rectangular(), delta_x),  # beam_width = scan.delta_x
    scan,
    sender,
    receiver,
    wave_data,
)
plot_visualization(
    plt, visualization, focal_point, sender.position, (-2.1, 2.1, -0.1, 5.1)
)


By using a window other than `Rectangular` we can get more "smoothed out" scanlines.

In [ ]:
from vbeam.apodization import Hamming

visualization = visualize_apodization(
    MLAApodization(Hamming(), delta_x * 10), scan, sender, receiver, wave_data
)
plot_visualization(
    plt, visualization, focal_point, sender.position, (-2.1, 2.1, -0.1, 5.1)
)


# Multiple Line Acquisition Apodization for Sector Scans
MLA scanlines are always in cartesian space, so in polar space, as in sector scans, they get distorted.

In [ ]:
from vbeam.scan import sector_scan


scan = sector_scan(np.linspace(-1, 1), np.linspace(0, 5))
sender = ElementGeometry(np.array([0.0, 0.0, 0.0]), 0.0, 0.0)
wave_data = WaveData(source=np.array([0.7, 0.0, 2.0]))

delta_x = scan.spacings[0]
visualization = visualize_apodization(
    MLAApodization(Rectangular(), delta_x), scan, sender, receiver, wave_data
)
plot_visualization(plt, visualization, extent=(-1, 1, 0, 5))
plt.xlabel("Angle")
plt.ylabel("Depth")


To see that the scanlines are actually straight we can perform a cartesian mapping of the image. The image is mapped to cartesian coordinates using interpolation, and since the points are more sparse the further out in a sector scan we get, we see that the line becomes blurry.

In [ ]:
sector_background_color = scan.cartesian_map(np.ones(scan.shape) * 0.2)
plot_visualization(
    plt,
    scan.cartesian_map(visualization) + sector_background_color,
    wave_data.source,
    sender.position,
    extent=scan.cartesian_bounds,
)


## Using a `Scan` Optimized for Cartesian Space
To get around the problem of sparse sampling of sector scans we can use a scan optimization method called `optimized_for_cartesian_space`. This converts the points themselves to cartesian coordinates before imaging and we get around the sampling problem. Notice how the line is equally sharp all the way through.

# TODO: Implement cartesian-space optimized sector scan

In [ ]:
visualization = visualize_apodization(
    MLAApodization(Hamming(), scan.delta_x * 4),
    scan.optimized_for_cartesian_space(),
    sender,
    receiver,
    wave_data,
)
plot_visualization(
    plt,
    visualization + sector_background_color,
    wave_data.source,
    sender.position,
    extent=scan.cartesian_bounds,
)
